In [1]:
from transformers.utils import logging

In [2]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer
import torch
DEVICE = torch.device("mps")

In [3]:
!export PYTORCH_ENABLE_MPS_FALLBACK=1

In [4]:
# Загружаем модель ruGPT от сбера
model_name_or_path = "../huggingface/hub/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path).to(DEVICE)

In [5]:
!ls "../huggingface/hub/rugpt3large_based_on_gpt2"

README.md               merges.txt              tokenizer_config.json
config.json             pytorch_model.bin       vocab.json
flax_model.msgpack      special_tokens_map.json


In [6]:
# prompt engineering 
text = "Вопрос: 'Сколько будет 2+2?' \n Ответ: " # работает
# text = " Вопрос: 'Сколько будет 3+3?' \n Ответ: 6 . \n Вопрос: 'Сколько будет 1+9?' \n Ответ: 10 . \n Вопрос: 'Сколько будет 4+2?' \n Ответ:" # ValueError
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
out = model.generate(input_ids, do_sample=False) 

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)

/opt/homebrew/lib/python3.11/site-packages/transformers/generation/utils.py:1141: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
/opt/homebrew/lib/python3.11/site-packages/transformers/generation/stopping_criteria.py:149: UserWarning: The operator 'aten::isin.Tensor_Tensor_out' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/pytorch/pytorch/pytorch/aten/src/ATen/mps/MPSFallback.mm:13.)
  is_done = torch.isin(input_ids[:, -1], self.eos_token_id.to(input_ids.device))



Вопрос: 'Сколько будет 2+2?' 
 Ответ: `2+2=4


1. %pip install transformers
2. загружаем Сберовскую модель.
3. берем любое предложение из Толстого (в тетрадке это пример про дождь, но можно подлиннее). 
4. пытаемся генерировать текст.
5. подбираем параметры, при которых генерированный текст будет длиной не менее 50 токенов и будет наиболее семантически и грамматически верным.
6. сдаем мне текст с описанием параметров.

In [10]:
# возьмем какой-нибудь текст
text = 'За окном дождь. Холодный и противный. Хочется'
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)

In [11]:
# если повторить запуск результат не изменится
# но тут уже нет зацикливания потому что модель смотрит дальше чем два токена в прошлое
out = model.generate(input_ids, do_sample=False, max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.




In [12]:
out = model.generate(input_ids, do_sample=True,  
                     top_k=0,  # про это параметр ниже
                     max_length=50)


generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


За окном дождь. Холодный и противный. Хочется уехать куда-нибудь.

В голове сложная, запутанная, путающаяся и противоречивая, но в тоже время такая ясная и продуманная, программа. Варианты сейчас, а завтра


## top_k
Этот параметр ограничивает количество слов, из которых мы семплируем. 10 - означает, что мы выбирает только из 10 самых вероятных слов. В ячейке выше мы поставили top_k = 0, потому что по умолчанию он стоит 50, а нам нужно было попробовать без него.

Чем больше top_k тем более случайный результат мы получим, но слишком низкий top_k может плохо сказаться на разнообразности.

In [13]:
for top_k in [1,3,10, 30, 100]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=top_k,  # про это параметр ниже
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with top_k - ", top_k)
    print(generated_text)
    print()

### text with top_k -  1
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with top_k -  3
За окном дождь. Холодный и противный. Хочется закрыть окно и не открывать. Но, если закрыть окно, то можно простудиться, а это не очень приятно.

Я не знаю, как можно было жить в такой стране.



### text with top_k -  10
За окном дождь. Холодный и противный. Хочется спрятаться под теплое одеяло и никуда больше не выходить.<s>
Друзья, кто подскажет, что за фильм, и как называется? +++
"Круто"<s>
Какую

### text with top_k -  30
За окном дождь. Холодный и противный. Хочется спать. А у двери сидит незнакомый мужчина с большими грустными глазами за которыми блестят слёзы. Он говорит, что его зовут Саша, он музыкант, и играет в джаз-банде, которая

### text with top_k -  100
За окном дождь. Холодный и противный. Хочется плюнуть ему в лицо. И чтобы потом при этом никого не ждало раскаяние за содеянное. И что

# Сэмплирование с Температурой
Еще случайность можно контролировать с помощью параметра, который называется температура. Температура изменяет распределение - при низком значении температуры вероятности переносятся от низких значений к высоким (распределение заостряется), а при высоком - вероятности переносятся от высоких значений к низким (распределение сглаживается).
https://camo.githubusercontent.com/6c20c0e34ce075adbf6bc0fd757b998e38319f819c5bbdca6e6a2b50b7f7b405/68747470733a2f2f6d69726f2e6d656469756d2e636f6d2f76322f726573697a653a6669743a313430302f312a794a55772d7a6b70546671645456654f4f374f4b6e512e706e67![image.png](attachment:image.png)

Нулевая температура означает, что мы на каждом шаге просто выбираем по argmax(), а очень большая температура будет приводить к полному рандому. Под конкретную задачу температуру нужно подбирать отдельно, можно начать с 0 и постепенно увеличивать, смотря на получаемое разнобразие.

(температурой это называется потому что формула взята из физических уравнений, где этот параметр действительно отвечает за температуру)

In [14]:
for temp in [0.001, 0.1, 0.2, 0.5, 0.7, 1., 1.5, 2., 3., 4., 5.]:

    out = model.generate(input_ids, do_sample=True,  
                     top_k=0, 
                     temperature=temp,
                     max_length=50)


    generated_text = list(map(tokenizer.decode, out))[0]
    print("### text with temp - ", temp)
    print(generated_text)
    print()

### text with temp -  0.001
За окном дождь. Холодный и противный. Хочется плакать.

— Я не могу, — говорит она. — Я не могу.

— Почему?

— Потому что я не могу.



### text with temp -  0.1
За окном дождь. Холодный и противный. Хочется плакать.

– Ты не плачь, – говорит мама. – Ты не плачь.

– Я не плачу, – говорю я. – Я

### text with temp -  0.2
За окном дождь. Холодный и противный. Хочется спрятаться под одеяло и закрыть глаза.

— Ты не спишь? — спрашивает мама.

— Нет, — отвечаю я.

— А я вот

### text with temp -  0.5
За окном дождь. Холодный и противный. Хочется плакать.

Я не знаю, почему я плачу.

Я не знаю, почему я плачу.

Я не знаю, почему я плачу.

Я не знаю, почему

### text with temp -  0.7
За окном дождь. Холодный и противный. Хочется под одеяло и не выходить из дома.<s>
Что делать, если я в депрессии? Как справиться с ней?
Отдыхать.<s>
Что делать, если я в депрессии? Как

### text with temp -  1.0
За окном дождь. Холодный и противный. Хочется в теплую, нежную постель. Д

# Beam Search
У подходов выше есть недостаток - на каждом шаге выбирается только 1 слово и этот выбор нельзя изменить, поэтому 1 неверно выбраное слово можно испортить весь текст и это сложно проконтролировать температурой и топ-к.

С этим может помочь beam-search (поиск пучком). Напомню, что в нем по сути происходит одновременная генерация нескольких текстов параллельно и в конце выбирается текст с наибольшей общей вероятностью. Генерировать все возможные варианты невозможно технически (потому что количество вариаций растет очень быстро), поэтому в beam search варианты отсеиваются на каждом шаге так, чтобы количество текущих вариантов было не больше N. Этот параметр N (размер пучка, beam size) настраивается, но поставить его слишком большим не получится, т.к. опять же будет слишком много комбинаций и это увеличит время генерации.

In [15]:
# beam search уже реализован в hg поэтому нужно только задать параметр num_beams
out = model.generate(input_ids, do_sample=True, num_beams=5, top_k=0, max_length=60)

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text.replace('<s>', ' '))


За окном дождь. Холодный и противный. Хочется в тепло, в уют, где тепло и уютно. 
А у вас бывает такое настроение, когда хочется, чтобы все было по-другому?
бывает. 
А у вас бывает такое настроение, когда хочется, чтобы все было


In [96]:
# возьмем модель поменьше, так как дообучение это обновление весов
# model_name_or_path = "sberbank-ai/rugpt3small_based_on_gpt2"
model_name_or_path = "../huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/a9307e696cd3c5b7f953ff4cb19d76a4d81821d5"
# model_name_or_path = "../huggingface/hub/rugpt3large_based_on_gpt2"
tokenizer = GPT2Tokenizer.from_pretrained(model_name_or_path)
model = GPT2LMHeadModel.from_pretrained(model_name_or_path, use_cache=False).to(DEVICE)

loading file vocab.json
loading file merges.txt
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading configuration file ../huggingface/hub/models--sberbank-ai--rugpt3small_based_on_gpt2/snapshots/a9307e696cd3c5b7f953ff4cb19d76a4d81821d5/config.json
Model config GPT2Config {
  "activation_function": "gelu_new",
  "architectures": [
    "GPT2LMHeadModel"
  ],
  "attn_pdrop": 0.1,
  "bos_token_id": 1,
  "embd_pdrop": 0.1,
  "eos_token_id": 2,
  "gradient_checkpointing": false,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "layer_norm_epsilon": 1e-05,
  "model_type": "gpt2",
  "n_ctx": 2048,
  "n_embd": 768,
  "n_head": 12,
  "n_inner": null,
  "n_layer": 12,
  "n_positions": 2048,
  "pad_token_id": 0,
  "reorder_and_upcast_attn": false,
  "resid_pdrop": 0.1,
  "scale_attn_by_inverse_layer_idx": false,
  "scale_attn_weights": true,
  "summary_

In [114]:
with open("train_dataset.txt") as f:
    text = f.read()
print(text[:50], "\n...\n", text[-50:], sep='')

len(text)

Том первый



Часть первая



I

– Eh bien, mon pr
...
ей к новой жизни, размягченной и ободренной душе.



1481336

In [149]:
# Полный тест 36Гб+ при обучении съедает :(
it = text.find('Том второй')
text[it-50:it+50]

'дежных раненых, был сдан на попечение жителей.\n\n\n\nТом второй\n\n\n\nЧасть первая\n\n\n\nI\n\nВ начале 1806-го '

In [150]:
cropped_text = text[it:]
len(cropped_text)

758325

In [151]:
it = cropped_text.find('Часть вторая')
print(cropped_text[it-50:it+50])

cropped_text = cropped_text[it:]
len(cropped_text)

оября догонять полк, который уже был в Польше.



Часть вторая



I

После своего объяснения с женой


636618

In [152]:
it = cropped_text.find('IV')
print(cropped_text[it-50:it+50])
print(len(cropped_text[:it]))

# режем чтобы в 32Гб MPS при обучении уместилось
cropped_text = cropped_text[:it]

ь радостию и умилением переполнявший его душу.



IV

Скоро после этого в темную храмину пришел за П
32732


In [145]:
from transformers import TextDataset, DataCollatorForLanguageModeling

# Сохраним обучающие данные в .txt файл 
train_path = 'train_dataset_cropped.txt'
with open(train_path, "w") as f:
     f.write(cropped_text)

# Создание датасета
train_dataset = TextDataset( tokenizer=tokenizer,file_path=train_path,block_size=64, 
                            overwrite_cache=True)
  
# специальный класс который будет подавать в модель данные в нужном ей виде
data_collator = DataCollatorForLanguageModeling(tokenizer=tokenizer, mlm=False)

Creating features from dataset file at 
Saving features into cached file cached_lm_GPT2Tokenizer_64_train_dataset_cropped.txt [took 0.302 s]


In [146]:
from transformers import Trainer, TrainingArguments
logging.set_verbosity(10)

training_args = TrainingArguments( 
    output_dir= "./finetuned",
    overwrite_output_dir=True,
    num_train_epochs=10, #100, 
    logging_steps=1,
    per_device_train_batch_size=32, 
    per_device_eval_batch_size=32,  
    gradient_accumulation_steps=32, 
    )


trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_dataset,
    optimizers = (torch.optim.AdamW(model.parameters(),lr=1e-5),None) # Optimizer and lr scheduler
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [147]:
trainer.train()

Currently training with a batch size of: 32
***** Running training *****
  Num examples = 126
  Num Epochs = 10
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 1,024
  Gradient Accumulation steps = 32
  Total optimization steps = 10
  Number of trainable parameters = 125,231,616


Step,Training Loss
1,0.452400
2,0.441300
3,0.433200
4,0.429300
5,0.423200
6,0.421300
7,0.416800
8,0.414600
9,0.414700
10,0.412000




Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=10, training_loss=0.425882613658905, metrics={'train_runtime': 34.771, 'train_samples_per_second': 36.237, 'train_steps_per_second': 0.288, 'total_flos': 41153495040000.0, 'train_loss': 0.425882613658905, 'epoch': 10.0})

In [153]:
DEVICE = torch.device("mps")

In [157]:
text = "'Вино? Объедение? Праздность? Леность? Горячность? Злоба? Женщины?'"
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        temperature=0.8,
                        top_k=50,
                        max_length=100, # при 300 уходит в swap
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


'Вино? Объедение? Праздность? Леность? Горячность? Злоба? Женщины?'Нерон, с трудом сдерживаясь, чтобы не броситься на свою горницу, но не успев, подошел к окну, отворил его и взглянул на улицу.

– Я слышал, ваше величество, – произнес он, – что вы с великим нетерпением ожидали приезда государя.

– Я счастлив видеть государя, – сказал король. – Он приехал.




In [158]:
torch.hub.get_dir()

'/Users/maxkozlov/.cache/torch/hub'

In [161]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=75,
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)


Дождь идет 
Александр Шмеман




Шмеман Александр

Дождь идет



Александр Шмеман 

Дождь идет 

Дождь идет. 

Дождь идет. 

Дождь идет. 

Дождь идет. 

Дождь идет. 

Дождь идет. 

Д


In [164]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=50,
                        max_length=75,
                        repetition_penalty=3.5 # штрафуем повторы
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Дождь идет 
С. С. Рахманинов


Писатель и поэт Сергей Рахманинов – один из самых ярких представителей русского авангарда начала XX века. Он родился в семье зажиточного помещика, рано осиротел и посвятил себя литературному творчеству. В начале 1920-х годов он становится одним из наиболее известных русских поэтов Серебряного века.
330


In [166]:
# всё бы хорошо, только Сергей Васильевич Рахманинов, 
# потомственный русский дворянин, гениальный пианист и композитор, 
# стал символом русской музыки во всем мире.

In [165]:
text = "Дождь идет "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=100,
                        max_length=75,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Дождь идет 
Дмитрий Быков

Стихотворение «Я иду» написано в конце 60-х годов прошлого века. Это одно из тех стихотворений, которые можно было бы назвать поэтическим произведением. Оно не имеет ничего общего с тем, что мы знаем сегодня о Пушкине и Лермонтове. В этом стихотворении нет ни одного стихотворения,
308


In [171]:
text = "Связан ли граф Пьер Безухов с масонами? "
input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=10,
                        max_length=75,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))


Связан ли граф Пьер Безухов с масонами?  И если это так, то как же он мог быть связан с масонством? 
 – Я не знаю.  Но мне кажется, что между ним и этим человеком есть какая-то связь; по крайней мере, я уверен в этом. 
 На следующий день после того, как все было готово к отъезду
279


In [173]:
text = """Связан ли граф Пьер Безухов с масонами?  И если это так, то как же он мог быть связан с масонством? 
 – Я не знаю.  Но мне кажется, что между ним и этим человеком есть какая-то связь; по крайней мере, я уверен в этом. 
 На следующий день после того, как все было готово к отъезду"""

input_ids = tokenizer.encode(text, return_tensors="pt").to(DEVICE)
model.eval()
with torch.no_grad():
    out = model.generate(input_ids, 
                        do_sample=True,
                        num_beams=5, top_k=10,
                        # max_length=75,
                        max_new_tokens=80,
                        repetition_penalty=3.5, # штрафуем повторы
                        no_repeat_ngram_size=2, # напрямую говорит, что нграммы такого размера не должны повторяться совсем
                        )

generated_text = list(map(tokenizer.decode, out))[0]
print()
print(generated_text)
print(len(generated_text))

Both `max_new_tokens` (=80) and `max_length`(=75) seem to have been set. `max_new_tokens` will take precedence. Please refer to the documentation for more information. (https://huggingface.co/docs/transformers/main/en/main_classes/text_generation)



Связан ли граф Пьер Безухов с масонами?  И если это так, то как же он мог быть связан с масонством? 
 – Я не знаю.  Но мне кажется, что между ним и этим человеком есть какая-то связь; по крайней мере, я уверен в этом. 
 На следующий день после того, как все было готово к отъезду, князь Голицын выехал из Петербурга на извозчике вместе со своей свитой во главе с княгиней Долгорукой, которая была тогда уже замужем за князем Петром Алексеевичем Вяземским (будущим великим князем Александром Михайловичем).  По дороге они проезжали мимо особняка графа Пьера Безухова, который был построен еще при Иване Васильевиче Грозном для великого князя
640
